In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [3]:
url = 'https://sutian.moe.edu.tw/zh-hant/huliok/106/'
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html')

In [9]:
url = 'https://sutian.moe.edu.tw/zh-hant/huliok/106/'
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser') 
table = soup.find_all('table')

# Find all <th> elements (table headers)
titles = soup.find_all('th')  

# Extract text from the <th> elements
table_titles = [title.text.strip() for title in titles]

# Create a DataFrame with the first four titles as headers
headers = table_titles[:4]
df = pd.DataFrame(columns=headers)

# Add an additional 'href' column
df['href'] = None

In [10]:
def scrapSinglePage(page):
    pageNum = str(page)
    url = 'https://sutian.moe.edu.tw/zh-hant/huliok/106/?iahbe=' + pageNum
    curPage = requests.get(url)
    soup = BeautifulSoup(curPage.text, 'html')
    table = soup.find_all('table')
    
    for row in table[:-1]:
        row_data = row.find_all('td')
        a_tag  = row.find('a')
        href_value = a_tag['href']
        individual_row_data = []
        for data in row_data:
            # Strip and clean the text
            cleaned_text = data.text.strip().replace('\n', '').replace('\t', '').replace('\r', '')
            
            # Find the first period and keep everything up to and including it
            period_index = cleaned_text.find('.')
            if period_index != -1:
                cleaned_text = cleaned_text[:period_index + 1]
        
            individual_row_data.append(cleaned_text)
            
        individual_row_data.append("https://sutian.moe.edu.tw/" + href_value)
        length = len(df)
        df.loc[length] = individual_row_data


In [11]:
for i in range(20):
    scrapSinglePage(i)

In [12]:
new_column_name = 'tags'
new_column_data = "slang"  # or you can use a list of values with the same length as the DataFrame

# Insert the new column at position 3 (between the third and fourth columns)
df.insert(4, new_column_name, new_column_data)

In [13]:
df.to_csv('output.csv', sep='|', index=False)